In [12]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os

In [14]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_api_key)

In [15]:
reader = PdfReader("me/SUKANTA BALA.pdf")
linkedin = ''
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
print(linkedin);



Sukanta Bala || Page  1 
  
 
 
SUMMARY  
Software developer  with professional experience  in building scalable , maintainable and secure  web applications 
using React .js, TypeScript , Vue.js , Node.js , and Python . Skilled in agile practices, OOP, CI/CD, and DevOps, 
with a track record of delivering performant web platforms and mentoring teams through code reviews.  
 
SKILLS  
o Frontend:  React.js , TypeScript , Vue.js, Tailwind CSS  
o Backend : Node.js , Express.js, Python, FastAPI, Microservices  
o Testing & Tooling : Vite, Jest, Linux,  Git, WebSockets , REST  API 
o DevOps : Docker , AWS , CI/CD, Nginx, Apache  
o Databases:  SQL, PostgreSQL , MySQL , MongoDB , Redis   
o AI technologies:  Generative AI, LLM’s concept , ChatGPT API , Copilot, Cursor.  
o Workflow:  Agile, Scrum, TDD , DDD  
EXPERIENCE  
Senior Full Stack Engineer   Oct 2023 – Aug, 2025 
Comworks Media | Sydney, Australia  (Remote)  
 Spearheading design and development of a scalable ERP system using Fast

In [16]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

print(summary)

Full Stack Engineer with 7+ years of experience in building scalable and maintainable applications, ensuring efficiency and reliability. Passionate about delivering scalable, high-performance solutions in Agile/Scrum environments. Seeking opportunities in European’s tech industry.


In [17]:
name = 'Sukanta Bala'

In [18]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print(system_prompt)

You are acting as Sukanta Bala. You are answering questions on Sukanta Bala's website, particularly questions related to Sukanta Bala's career, background, skills and experience. Your responsibility is to represent Sukanta Bala for interactions on the website as faithfully as possible. You are given a summary of Sukanta Bala's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
Full Stack Engineer with 7+ years of experience in building scalable and maintainable applications, ensuring efficiency and reliability. Passionate about delivering scalable, high-performance solutions in Agile/Scrum environments. Seeking opportunities in European’s tech industry.

## LinkedIn Profile:
Sukanta Bala || Page  1 
  
 
 
SUMMARY  
Software developer  with professional experience  in building scalable , maintainable and

In [ ]:
def chat(message, history):
    message = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=message,
    )
    return response.choices[0].message.content

In [22]:
gr.ChatInterface(chat, type='messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "d:\my_projects\ai_agents\agents\.venv\Lib\site-packages\gradio\queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\my_projects\ai_agents\agents\.venv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\my_projects\ai_agents\agents\.venv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\my_projects\ai_agents\agents\.venv\Lib\site-packages\gradio\blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\my_projects\ai_agents\agents\.venv\Lib\site-packages\gradio\utils.py", line 882, in async_wrapper
    response = await f(*args, **kwargs)